In [1]:
def analysing_the_system(system):
    """
    Функция находит нелинейное уравнение в системе и отделяет его от линейных
    Линейные уравнения в свою очередь помещаются в специальный список
    """
    hardest_equation = ''
    lst_easy = []
    for equation in system:
        if ('^2' or '^3' or '^4') in equation:
            hardest_equation = equation
        else:
            lst_easy.append(equation)

    return(hardest_equation, lst_easy)

In [2]:
def check_float(el):
    """
    Функция проверяет, может ли данная перенная быть переведенной в float из str
    """
    try:
        el = float(el)
        return(True)
    except:
        return(False)

In [3]:
def pretty_equation(equation):
    '''
    Перенос всех слагаемых на одну сторону
    Подсчет подобных слагаемых
    '''
    lst_equation = list(equation.split(' '))
    
    if lst_equation[0] != '-':                        #добавляем плюс на первое место
        lst_equation.insert(0,'+')
    
    for i, el in enumerate(lst_equation):             #добавляем плюс на первое место после равно
        if el == '=' and lst_equation[i+1] != '0' and lst_equation[i+1] != '-':
            lst_equation.insert(i+1,'+')
    
    for i, el in enumerate(lst_equation):             #переводим числа из str в float
        if check_float(el):
            lst_equation[i] = float(el)
    
    
    buf_fv = []
    for el in lst_equation:
        if el != 'free_var':
            buf_fv.append(el)
    lst_equation = buf_fv
            

        
    
    lst_equation_buf2 = []                           #перенос всех слагаемых после равно на левую сторону
    for i, el in enumerate(lst_equation):
        if el == '=' :
            for ind, after in enumerate(lst_equation[i::]):
                if after == '-':
                    lst_equation_buf2.append('+')
                    lst_equation_buf2.append(lst_equation[i::][ind+1])
                if after == '+':
                    lst_equation_buf2.append('-')
                    lst_equation_buf2.append(lst_equation[i::][ind+1])
            
            
            lst_equation_buf2.append('=')
            lst_equation_buf2.append(0)
            
            break
            
        else:
            lst_equation_buf2.append(el)
    lst_equation = lst_equation_buf2

    
    
    lst_equation_buf = []                             #получаем посчитанные коэффициенты
    for i in range(len(lst_equation)-1):
        if lst_equation[i] == '+' and type(lst_equation[i+1]) is float:
            lst_equation_buf.append(1*lst_equation[i+1])
        elif lst_equation[i] == '+' and type(lst_equation[i+1]) is str:
            lst_equation_buf.append(1)
        elif lst_equation[i] == '-' and type(lst_equation[i+1]) is float:
            lst_equation_buf.append(-1*lst_equation[i+1])
        elif lst_equation[i] == '-' and type(lst_equation[i+1]) is str:
            lst_equation_buf.append(-1)
        elif type(lst_equation[i]) is float:
            pass
        else:
            lst_equation_buf.append(lst_equation[i])  

    lst_equation = lst_equation_buf 

    
    unique_var = []                                  #считаем подобные
    for el in lst_equation:
        if type(el) == str and el != '=' and el not in unique_var:
            unique_var.append(el)
    
    dict_buf_var = {}    
    for var in unique_var:
        dict_buf_var[var] = []
        for i, el in enumerate(lst_equation):
            if el == var:
                dict_buf_var[var].append(lst_equation[i-1])
        dict_buf_var[var] = sum(dict_buf_var[var]) 
    
    lst_equation_rezult = []
    for k, v in dict_buf_var.items():
        lst_equation_rezult.append(v)
        lst_equation_rezult.append(k)

    
    f_var = 0                                        #добавляем свободный член
    for i, el in enumerate(lst_equation):
        if (type(el)== float):
            if i == len(lst_equation)-1:
                f_var += el
            else:
                if type(lst_equation[i+1]) is float: 
                    f_var += el
                if type(lst_equation[i+1]) is int: 
                    f_var += el
                if lst_equation[i+1] == '=':
                    f_var += el
    lst_equation_rezult.append(f_var)   
    
    lst_equation = lst_equation_rezult
            
    return(lst_equation)

In [4]:
def check_alright(lst,system):
    """
    Проверка, что кол-во переменных совпадает с кол-вом уравнений в системе
    """
    lst_all_var = []
    for equation in lst:
        for ind, el in enumerate(equation):
            if type(el) is str and '^' in el:
                if list(el.split('^'))[0] not in lst_all_var:
                    lst_all_var.append(list(el.split('^'))[0])
            elif type(el) is str and el not in lst_all_var:
                lst_all_var.append(el)

    if len(lst_all_var) == len(system):
        return(True)
    else:
        return(False)

In [5]:
def matrix_for_easy(lst_easy):
    """
    Приводим линейные уравнения в матричный вид
    """
    lst_all_var = []
    for equation in lst_easy:
        for el in equation:
            if type(el) is str and '^' not in el:
                if el not in lst_all_var:
                    lst_all_var.append(el)
                elif list(el.split('^'))[0] not in lst_all_var:
                    lst_all_var.append(list(el.split('^'))[0])
                else:
                    pass
    matrix = []
    
    for equation in lst_easy:
        buf_lst = []
        dict_matr = {}
        for el in lst_all_var:
            dict_matr[el] = 0
        for i,var in enumerate(equation):
            if var in dict_matr.keys():
                dict_matr[var] = equation[i-1]
            elif i + 1 == len(equation) and type(var) is float:    #можно было через or
                dict_matr['free_var'] = var
            elif i + 1 == len(equation) and type(var) is int:
                dict_matr['free_var'] = var
                
        for v in dict_matr.values():
            buf_lst.append(v)
        matrix.append(buf_lst)
    
    return(matrix, lst_all_var)

In [6]:
def check_continue(matrix, n):
    """
    Функция проверяет, можно ли поменять некоторые уравнения местами, чтобы продолжить приводить матрицу к ступенчатому виду
    """
    boolean = False
    for line in matrix[n::]:
        if line[n] != 0:
            boolean = True
    return(boolean)

In [7]:
def rearrangement(matrix,n):
    """
    Функция меняет уравнения местами, чтобы продолжить приводить матрицу к ступенчатому виду
    """
    for ind, line in enumerate(matrix):
        if line[n] != 0:
            if ind > n: 
                matrix[ind], matrix[ind-1] = matrix[ind-1], matrix[ind]
    return(matrix)      

In [8]:
def matrix_standart_form(matrix):
    """
    Приводим матрицу к ступенчатому виду
    """
    count = 0
    while count < len(matrix)-1:

        if matrix[count][count] != 0:                      #делим первую строчку на "нужный" элемент в данной строчке
            buf = matrix[count][count]
            for i, el in enumerate(matrix[count]):
                matrix[count][i] = el / buf
        
        if matrix[count][count] == 1:
            for ind, line in enumerate(matrix[count+1::]):
                buf2 = line[count]
                for i, el in enumerate(line):
                    line[i] = el - buf2*matrix[count][i]
        
        buf_for_exit = 0                          
        if matrix[count][count] == 0:
            if check_continue(matrix, count):            #проверяем, можем ли мы переставить уравнения местами
                matrix = rearrangement(matrix,count)     #переставлем уравнения местами
                
                
                
                buf = matrix[count][count]              #проводим те же самые действия
                for i, el in enumerate(matrix[count]):
                    matrix[count][i] = el / buf
                
                
                for ind, line in enumerate(matrix[count+1::]):
                    buf2 = line[count]
                    for i, el in enumerate(line):
                        line[i] = el - buf2*matrix[count][i]
                
            else:
                buf_for_exit = 1
                
        
        if buf_for_exit == 0:
            count += 1
        elif buf_for_exit == 1:
            break
        
    return(matrix, buf_for_exit)

In [9]:
def sub_for_sub(line,var, lst_all_var):
    """
    Вспомогательная функция для выражения переменных в приведенной матрице
    """
    buf = 0
    for i, el in enumerate(line):
        if el != 0:
            buf = el
            break
    
    for i, el in enumerate(line):
        if type(el) is int or float:
            line[i] = el/buf
    
    rez = list(zip(lst_all_var,line))  #[('x', -0.0), ('y', 1.0), ('z', -0.6666666666666666), ('free_var', 3.6666666666666665)]
   
    rez_str = ''
    for ind, cort in enumerate(rez):
        if cort[0] != var and cort[1] != 0 :
            if cort[1] < 0:
                rez_str += f'+ {round(-1*cort[1],2)} {cort[0]} '
            elif cort[1] > 0:
                rez_str += f'- {round(cort[1],2)} {cort[0]} '
    
    return rez_str

In [10]:
def final_sub(str_before_value, current_var, current_value):
    """
    Функция подставляет предыдущее выражение переменной в настоящую
    Предыдущая переменная и ее значение - dict_before
    Настоящая переменная - current_var
    Настоящее значение переменной - current_value
    """
    lst_current_value = list(current_value.split(' '))
    
    for key, value in str_before_value.items():
        before_var = key
        before_value = value
    lst_before_value = list(before_value.split(' '))
        
    rezult_str = ''
    for i, el in enumerate(lst_current_value):
        if el == before_var:
            if lst_current_value[i-2] == '-':
                buf_sign = -1
            if lst_current_value[i-2] == '+':
                buf_sign = 1
            buf_mult = float(lst_current_value[i-1])*buf_sign
            
            for i_b , el_b in enumerate(lst_before_value):
                if el_b == '+':
                    
                    if 1*buf_mult*float(lst_before_value[i_b+1]) > 0:
                        rezult_str += f'+ {abs(1*buf_mult*float(lst_before_value[i_b+1]))} {(lst_before_value[i_b+2])} '
                    else:
                        rezult_str += f'- {abs(1*buf_mult*float(lst_before_value[i_b+1]))} {(lst_before_value[i_b+2])} '
                    
                if el_b == '-':
                    if -1*buf_mult*float(lst_before_value[i_b+1]) > 0:
                        rezult_str += f'+ {abs(1*buf_mult*float(lst_before_value[i_b+1]))} {(lst_before_value[i_b+2])} '
                    else:
                        rezult_str += f'- {abs(1*buf_mult*float(lst_before_value[i_b+1]))} {(lst_before_value[i_b+2])} '
    
        elif el == '':
            pass   
        elif (el != before_var) and (el != '+') and (el != '-') and (not check_float(el)):
            rezult_str += f'{lst_current_value[i-2]} {lst_current_value[i-1]} {lst_current_value[i]}' 
    return(rezult_str)

In [11]:
def substitution(matrix, lst_all_var):
    """
    Функция выражает все перменные в системе из линейных уранений через одну базисную
    """
    dict_rezult = {}
    lst_all_var.append('free_var')
    for ind, line in enumerate(matrix[::-1]):
        for i, el in enumerate(line):
            if el != 0 :
                dict_rezult[lst_all_var[i]] = sub_for_sub(line,lst_all_var[i],lst_all_var)
                break
    
    new_dict_rezult = {}
    buf_count = 0
    for var, value in dict_rezult.items():  #так как мы имеем дело сос ступенчатым видом, то мы можем так сделать
        if buf_count == 0:
            new_dict_rezult[var] =pretty_equation(value) #бессмысленно
            var_before = {var:value} 
        else:
            new_dict_rezult[var] = pretty_equation(final_sub(var_before,var,value))
            var_before = {var:value}
        buf_count += 1
    

    return(new_dict_rezult)

In [12]:
def squaring(lst_to_square):
    """
    Функция возведения в квадрат
    """
    lst_rezult = []
    if len(lst_to_square) == 1:             #для случая x=5
        lst_rezult.append(lst_to_square[0]**2)
    elif len(lst_to_square) == 3:           #для случая x= 5 y - 2
        lst_rezult.append(round((lst_to_square[0]**2),4))
        lst_rezult.append(f'{lst_to_square[1]}^2')
        lst_rezult.append(round((2*lst_to_square[0]*lst_to_square[2]),2))
        lst_rezult.append(f'{lst_to_square[1]}')
        lst_rezult.append(round((lst_to_square[2]**2),4))
    else:
        print('Для возведения в квадрат было передано выражение не соответсвующее условию')
        
    return lst_rezult

In [13]:
def cubing(lst_to_cube):
    """
    Функция возведения в куб
    """
    lst_rezult = []
    if len(lst_to_cube) == 1:
        lst_rezult.append(lst_to_cube[0]**3)
    elif len(lst_to_cube) == 3:
        lst_rezult.append(round((lst_to_cube[0]**3),5))
        lst_rezult.append(f'{lst_to_cube[1]}^3')
        lst_rezult.append(round((3*(lst_to_cube[0]**2)*lst_to_cube[2]),4))
        lst_rezult.append(f'{lst_to_cube[1]}^2')
        lst_rezult.append(round((3*lst_to_cube[0]*(lst_to_cube[2]**2)),4))
        lst_rezult.append(f'{lst_to_cube[1]}')
        lst_rezult.append(round((lst_to_cube[2]**3),5))
    else:
        print('Для возведения в куб было передано выражение не соответсвующее условию')
        
    return lst_rezult

In [14]:
def power_four(lst_to_four):
    """
    Функция возведения выражения в четвертую степень
    """
    lst_rezult = []
    if len(lst_to_four) == 1:
        lst_rezult.append(lst_to_four[0]**4)
    elif len(lst_to_four) == 3:
        lst_rezult.append(round((lst_to_four[0]**4),10))
        lst_rezult.append(f'{lst_to_four[1]}^4')
        lst_rezult.append(round((4*(lst_to_four[0]**3)*lst_to_four[2]),6))
        lst_rezult.append(f'{lst_to_four[1]}^3')
        lst_rezult.append(round((6*(lst_to_four[0]**2)*(lst_to_four[2]**2)),4))
        lst_rezult.append(f'{lst_to_four[1]}^2')
        lst_rezult.append(round((4*(lst_to_four[2]**3)*lst_to_four[0]),6))
        lst_rezult.append(f'{lst_to_four[1]}')
        lst_rezult.append(round((lst_to_four[0]**4),10))
    else:
        print('Для возведения в четвертую степень было передано выражение не соответсвующее условию')
        
    return lst_rezult

In [15]:
def sub_into_hard(hard_equation, dict_with_var):
    """
    Функция общего подставления выраженных переменных в нелинейное уравнение
    """
    not_final_lst = []
    for i, el in enumerate(hard_equation):
        if type(el) is str:
            if '^' in el:
                for var, value in dict_with_var.items():
                    boolean = 0
                    if list(el.split('^'))[0] == var:
                        if '^2' in el:
                            not_final_lst.append(squaring(value))
                            boolean = 1
                        elif '^3' in el:
                            not_final_lst.append(cubing(value))
                            boolean = 1
                        elif '^4' in el:
                            not_final_lst.append(power_four(value))
                            boolean = 1
                        else:
                            pass
                    if boolean == 1:
                        break
                if boolean == 0:
                    not_final_lst.append(el)
                elif boolean == 1:
                    pass
            else:
                for var, value in dict_with_var.items():
                    if var == el:
                        not_final_lst.append(value)
                    
            
        else:
            not_final_lst.append(el)
            
                    
                    
            
                
                
    
    return not_final_lst

In [16]:
def pretty_lst_for_final_rezult(lst):
    """
    Функция считает коэффициенты после возведения в степень
    """
    lst_rezult = []
    for i, el in enumerate(lst):
        if (type(el) is float or int) and i != len(lst)-1 and type(lst[i+1]) is list:
            for i_l, el_l in enumerate(lst[i+1]):
                if type(el_l) is float or int:
                   
                    lst[i+1][i_l] = lst[i+1][i_l]*el
            lst_rezult.append(lst[i+1])
        elif type(el) is float or int and i == len(lst)-1:
            lst_rezult.append(el)
        elif type(el) is str:
            lst_rezult.append(el)
        else:
            pass
                
   
    return(lst_rezult)

In [17]:
def from_lst_to_str(lst):
    """
    Перевод списка в строку
    """
    str_rez = ''
    for i, el in enumerate(lst):
        if type(el) is str:
           
            str_rez += f'{el} '
        elif type(el) is list:
            str_rez += from_lst_to_str(el)
        elif type(el) is float or int:
            if el >= 0:
                str_rez += f'+ {abs(el)} '
            else:
                str_rez += f'- {abs(el)} '
        else:
            pass
    
    return str_rez     

In [18]:
def solve_quadratic_equation(lst):
    """
    Функция решения квадратных уравнений
    """
    
    for el in lst:
        if type(el) is str and ('^2' in el):
            basis_parametr = list(el.split('^'))[0]
    
    lst_rez = []
    if len(lst) == 5:
        
        if '^2' not in lst[1]:                               #случай 5 - 6x - 7x^2
            buf_lst = [0]*5
            for i, el in enumerate(lst):
                if type(el) is str and '^2' in el:
                    buf_lst[0] = lst[i-1]
                    buf_lst[1] = lst[i]                #el
                elif type(el) is str :
                    buf_lst[2] = lst[i-1]
                    buf_lst[3] = lst[i]
                elif (type(el) is float or int) and (i == 4 or (type(lst[i+1]) is float or int)):
                    buf_lst[4] = lst[i]
                else:
                    pass
            lst = buf_lst      
        
        a = lst[0] 
        b = lst[2]
        c = lst[4]
        D = b**2 - 4*a*c
        if D > 0:
            rez1 = round((-b + D**0.5)/(2*a), 5)
            rez2 = round((-b - D**0.5)/(2*a), 5)
            lst_rez.append(rez1)
            lst_rez.append(rez2)
            return (lst_rez, basis_parametr)
        elif D == 0:
            rez = round((-b/(2*a)), 5)
            lst_rez.append(rez)
            return (lst_rez, basis_parametr)
        else:
            raise ValueError('Слишком трудное квадратное уравнение необходимо решить')
    else:
        raise ValueError('Слишком трудное квадратное уравнение необходимо решить')

In [19]:
def solve_mini_quadratic(lst):
    """
    Функция находит корни квадратного уравнения, где отсутсвует переменная в 1 степени
    """
    assert len(lst) == 3, 'Проверь введеное квадратное уравнение'
    
    for el in lst:
        if type(el) is str and ('^2' in el):
            basis_parametr = list(el.split('^'))[0]
    lst_rez = []
            
    if (lst[0] > 0 and lst[2] < 0) or (lst[2] > 0 and lst[0] < 0):
        rez1 = (-1*lst[2]/lst[0])**0.5
        rez2 = -1*(-1*lst[2]/lst[0])**0.5
        lst_rez.append(rez1)
        lst_rez.append(rez2)
        return (lst_rez, basis_parametr)
    else:
        raise ValueError('Слишком трудное квадратное уравнение необходимо решить')

In [20]:
# def solve_cube_equation(lst):
#     """
#     Функция решения кубических уравнений
#     """
#     print('зашли в кубическое уравнение')
    
#     for el in lst:
#         if type(el) is str and ('^3' in el):
#             basis_parametr = list(el.split('^'))[0]
    
#     count1 = 0
#     count2 = 0
#     count3 = 0
#     buf_0_k = 0
#     for i,el in enumerate(lst):
#         if type(el) is str and '^3' in el:
#             count3 += 1
#             buf_3 = el
#             buf_3_k = lst[i-1]
#         elif type(el) is str and '^2' in el:
#             count2 += 1 
#             buf_2 = el
#             buf_2_k = lst[i-1]
#         elif type(el) is str:
#             count1 += 1
#             buf_1 = el
#             buf_1_k = lst[i-1]
#         elif type(el) is float or int and (i == 6 or (type(lst[i+1] is float or int))):
#             buf_0_k = el
#         else:
#             pass
        
#     if count1 == 0:
#         buf_1 = basis_parametr
#         buf_1_k = 0
#     if count2 == 0:
#         buf_2 = f'{basis_parametr}^2'
#         buf_2_k = 0
        
#     lst[0] = buf_3_k
#     lst[1] = buf_3
#     lst[2] = buf_2_k
#     lst[3] = buf_2
#     lst[4] = buf_1_k
#     lst[5] = buf_1
#     lst[6] = buf_0_k
    
#     print(f'перед подсчетом куб ур имеет вид {lst}')
#     lst_rez = []
#     if len(lst) == 7:            #лишнее
        
#         if lst[0] != 1:
#             buf = lst[0] 
#             lst[0] = lst[0]/buf
#             lst[2] = lst[2]/buf         #если коэфф. при старшей степени не равен 1, то делим все остальные коэфф. на него
#             lst[4] = lst[4]/buf
#             lst[6] = lst[6]/buf
            
#         dividers_free_var = []
        
#         print(f'после коэфф ур имеет вид {lst}')
        
#         start_point = 0.01
#         while start_point <= abs(lst[6]):                       #ищем делители свободного члена
#             if -0.05 <= lst[6] % start_point <= 0.05:
#                 dividers_free_var.append(start_point)
#                 dividers_free_var.append(-1*start_point)
#             start_point += 0.01
            
#         print(f'изначально dividers_free_var {dividers_free_var}')
#         for divider in dividers_free_var:    #проверяем, есть ли корни среди них
#             print(lst[0]*(divider**3)+lst[2]*(divider**2)+lst[4]*divider+lst[6])
#             if -0.5 <= lst[0]*(divider**3)+lst[2]*(divider**2)+lst[4]*divider+lst[6] <= 0.5:
#                 lst_rez.append(divider)
#         print(f'1 lst_rez {lst_rez}')
        
        
#         if len(lst_rez) > 3:                              #на случай, если корней получилось больше 3
#             buf_dict = {}
#             for divider in lst_rez:
#                 buf_dict[divider] = lst[0]*(divider**3)+lst[2]*(divider**2)+lst[4]*divider+lst[6]
#             buf_dict = sorted(buf_dict, key=buf_dict.get, reverse=False)
            
            
            
#             lst_rez = []
#             k = 0
#             for divider, value in buf_dict.items():
#                 k += 1
#                 if k <= 2:
#                     lst_rez.append(divider)
    
#         if len(lst_rez) == 0:
#             return {'ans':'Данное кубическое уравнение не имеет корней'}
#         else:
#             return (lst_rez, basis_parametr)
        
#     else:
#         return {'ans':'Проверь кубическое уравнение поступившее на обработку'}
        

In [21]:
import math               #для вычисления косинусов и арккосинусов

In [22]:
def arch(x):
    """
    arch
    """
    rez = math.log10(x+(x**2 - 1)**0.5)
    return rez

In [23]:
def arsh(x):
    """
    arsh
    """
    rez = math.log10(x+(x**2 + 1)**0.5)
    return rez

In [24]:
def ch(x):
    """
    ch
    """
    rez = (math.exp(x) + math.exp(-x))/2
    return rez

In [25]:
def sh(x):
    """
    sh
    """
    rez = (math.exp(x) - math.exp(-x))/2
    return rez

In [26]:
def solve_cube_equation(lst):
    """
    Функция решения кубических уравнений. Метод Виета-Кардана
    """
    for el in lst:
        if type(el) is str and ('^3' in el):
            basis_parametr = list(el.split('^'))[0]
    
    count1 = 0
    count2 = 0
    count3 = 0
    buf_0_k = 0
    for i,el in enumerate(lst):
        if type(el) is str and '^3' in el:
            count3 += 1
            buf_3 = el
            buf_3_k = lst[i-1]
        elif type(el) is str and '^2' in el:
            count2 += 1 
            buf_2 = el
            buf_2_k = lst[i-1]
        elif type(el) is str:
            count1 += 1
            buf_1 = el
            buf_1_k = lst[i-1]
        elif type(el) is float or int and (i == 6 or (type(lst[i+1] is float or int))):
            buf_0_k = el
        else:
            pass
        
    if count1 == 0:
        buf_1 = basis_parametr
        buf_1_k = 0
    if count2 == 0:
        buf_2 = f'{basis_parametr}^2'
        buf_2_k = 0
        
    lst[0] = buf_3_k
    lst[1] = buf_3
    lst[2] = buf_2_k
    lst[3] = buf_2
    lst[4] = buf_1_k
    lst[5] = buf_1
    lst[6] = buf_0_k
    
    if lst[0] != 1:
        buf = lst[0] 
        lst[0] = lst[0]/buf
        lst[2] = lst[2]/buf         #если коэфф. при старшей степени не равен 1, то делим все остальные коэфф. на него
        lst[4] = lst[4]/buf
        lst[6] = lst[6]/buf
    #Метод Виета-Кардана:
    d = lst[0]
    a = lst[2]
    b = lst[4]
    c = lst[6]
    
    Q = (a**2-3*b)/9                             #Формулы из интернета
    R = (2*(a**3) - 9*a*b + 27*c)/54
    S = Q**3 - R**2
    
    lst_rez = []
    if S > 0:
        q = 1/3*math.acos(R/(Q**(1/3)))
        try:
            rez1 = -2*(Q**0.5)*math.cos(q) - a/3
            lst_rez.append(rez1)
        except:
            pass
        try:
            rez2 = -2*(Q**0.5)*math.cos(q - (2/3)*math.pi)
            lst_rez.append(rez2)
        except:
            pass
        try:
            rez3 = -2*(Q**0.5)*math.cos(q + (2/3)*math.pi)
            lst_rez.append(rez3)
        except:
            pass
        
        
    elif S < 0:
        if Q > 0:
            if R > 0:
                q = (1/3)*arch( (abs(R)) / ((Q**3)**(0.5)) )
                rez = -2*1*(Q**0.5)*ch(q) - a/3
            if R < 0:
                q = (1/3)*arch( (abs(R)) / ((Q**3)**(0.5)) )
                rez = 2*1*(Q**0.5)*ch(q) - a/3
        if Q < 0:
            q = (1/3)*arsh( (abs(R)) / ((abs(Q)**2)**(1/3)) )
            if R > 0:
                rez = -2*1*(abs(Q)**0.5)*sh(q) - a/3
            if R < 0:
                rez = 2*1*(abs(Q)**0.5)*sh(q) - a/3
        lst_rez.append(rez)
    else:
        rez1 = -2*(R**(1/3))-a/3
        rez2 = (R**(1/3))-a/3
        lst_rez.append(rez1)
        lst_rez.append(rez2)
     
    if len(lst_rez) != 0:    
        return (lst_rez,basis_parametr)
    else:
        raise ValueError('Слишком трудное кубическое уравнение необходимо решить')
            


In [27]:
def solve_biquadratic_equation(lst):
    """
    Функция решения биквадратных уравнений
    """
    for el in lst:
        if type(el) is str and ('^4' in el):
            basis_parametr = list(el.split('^'))[0]
            
    lst_rez = []
    if len(lst) == 5 or 3:
        
        if '^4' not in lst[1]:                               #случай 5 - 6x^2 - 7x^4
            buf_lst = [0]*5
            for i, el in enumerate(lst):
                if type(el) is str and '^4' in el:
                    buf_lst[0] = lst[i-1]
                    buf_lst[1] = lst[i]                #el
                elif type(el) is str and '^2' in el:
                    buf_lst[2] = lst[i-1]
                    buf_lst[3] = lst[i]
                elif (type(el) is float or int) and (i == 4 or (type(lst[i+1]) is float or int)):
                    buf_lst[4] = lst[i]
                else:
                    pass
            lst = buf_lst 
            
        a = lst[0]
        b = lst[2]
        c = lst[4]
        D = b**2 - 4*a*c
        if D > 0:
            rez1 = round((-b + D**0.5)/(2*a), 5)
            rez2 = round((-b - D**0.5)/(2*a), 5)
            
            if rez1 > 0:
                lst_rez.append(rez1**0.5)
                lst_rez.append(-1*rez1**0.5)
            if  rez2 > 0:
                lst_rez.append(rez2**0.5)
                lst_rez.append(-1*(rez2**0.5))
            if rez1 == 0:
                lst_rez.append(rez1)
            if rez2 == 0:
                lst_rez.append(rez2)
            
            return (lst_rez, basis_parametr)
        elif D == 0:
            rez = round((-b/(2*a)), 5)
            
            if rez > 0:
                lst_rez.append(rez**0.5)
                lst_rez.append(-1*(rez**0.5))
            if rez == 0:
                lst_rez.append(rez)
            return (lst_rez, basis_parametr)
        else:
            raise ValueError('Слишком трудное биквадратное уравнение необходимо решить')
    else:
        raise ValueError('Слишком трудное биквадратное уравнение необходимо решить')
        
        

In [28]:
def getting_answer(hard_final_lst, dict_with_var):
    """
    1) Функция оределяет вид приведенного нелинейного уравнения (квадратное, кубическое, биквадратное или линейное)
    2) Создает систему (совокупность) решений
    """
    count_4 = 0
    count_3 = 0
    count_2 = 0
    count_1 = 0
    for el in hard_final_lst:
        if type(el) is str and '^2' in el:
            count_2 += 1
        elif type(el) is str and '^3' in el:
            count_3 += 1
        elif type(el) is str and '^4' in el:
            count_4 += 1
        elif type(el) is str:
            count_1 += 1
        else:
            pass
    
    rezult_of_basis_var = []
    
    if count_1 == 1 and count_2 == 1 and count_3 == 0 and count_4 == 0:       #квадратное уравнение
        rezult_of_basis_var, var_basis = solve_quadratic_equation(hard_final_lst)
        boolean = True
    elif count_1 == 0 and count_2 == 1 and count_3 == 0 and count_4 == 1:       #биквадратное уравнение
        rezult_of_basis_var, var_basis = solve_biquadratic_equation(hard_final_lst)
        boolean = True
#     elif count_1 == 1 and count_2 == 1 and count_3 == 1 and count_4 == 1:
#         boolean = False
    elif count_3 == 1 and count_4 == 0 :                                          #кубическое уравнение
        rezult_of_basis_var, var_basis = solve_cube_equation(hard_final_lst)
        boolean = True
    elif count_1 == 0 and count_2 == 1 and count_3 == 0 and count_4 == 0:       #квадратное уравнение без x
        rezult_of_basis_var, var_basis = solve_mini_quadratic(hard_final_lst)
        boolean = True
    elif count_1 == 1 and count_2 == 1 and count_3 == 1 and count_4 == 1:
        boolean = True
        return {'ans':'Программа не может решить данную систему, так как финальное уравнение содержит и 1, и 2, и 3, и 4 степени'}
    elif hard_final_lst == [0]:
        return dict_with_var                                       
        boolean = False
        
    if boolean:
        answer_dict = {}
        count_buf = 0
        while count_buf <= len(rezult_of_basis_var) - 1:
            buf_answer_dict = {}
            buf_answer_dict[var_basis] = rezult_of_basis_var[count_buf]
            
            for var, value in dict_with_var.items():
                summa = 0
                for ind, el in enumerate(value):
                    if type(el) is str:
                        summa += buf_answer_dict[var_basis]
                    elif type(el) is float or int:
                        summa += el
                buf_answer_dict[var] = summa
                        
            answer_dict[count_buf+1] = buf_answer_dict
            count_buf += 1
        
        return (answer_dict)
    else:
        raise ValueError('Что-то пошло не так на определении типа трудного уравнения')


In [29]:
def solve(system):
    """
    Функция выводит решение системы уравнений
    """
    hard_equation, lst_easy = analysing_the_system(system)
    
    hard_equation = pretty_equation(hard_equation)
    for i,eq in enumerate(lst_easy):
        lst_easy[i] = pretty_equation(eq) 
    
    lst_all_equation = []
    for line in lst_easy:
        lst_all_equation.append(line)
    lst_all_equation.append(hard_equation)
    
    if check_alright(lst_all_equation,system):
        matrix = []
        lst_var = []
        matrix, lst_var = matrix_for_easy(lst_easy)
        for_exit = 0
        matrix, for_exit = matrix_standart_form(matrix)
        print(matrix, for_exit)
        if for_exit == 1:
            return 'Матрица, составленная из линейных уравнений не может быть приведена к ступенчатому виду'
        else:
            dict_for_substitution = {}
            dict_for_substitution = substitution(matrix, lst_var) #получаем выраженные переменные
            
            print(f'dict_for_substitution {dict_for_substitution}')

            lst_for_final_rezult = [] 
            lst_for_final_rezult = sub_into_hard(hard_equation, dict_for_substitution) #возводим в степени выражения
           
            lst_for_final_rezult = pretty_lst_for_final_rezult(lst_for_final_rezult) #считаем коэффициенты
            print(f'lst_for_final_rezult {lst_for_final_rezult}')
            lst_for_final_rezult = pretty_equation(from_lst_to_str(lst_for_final_rezult)) #приводим подобные
            
            answer = {}
            answer = getting_answer(lst_for_final_rezult, dict_for_substitution)
            
            return (f'ОТВЕТ: {answer}')
    
    else:
        return 'Колл-во переменных не совпадает с кол-вом уравнений'
    

In [30]:
system = [
    '- 8 + 4 x + y = 5',
    '2 x + z - y + 1 = 8',
    'x^3 + y^2 = 9 + 5'
]
solve(system)


[[1.0, 0.25, 0.0, -3.25], [0.0, -1.5, 1.0, -0.5]] 0
dict_for_substitution {'y': [0.67, 'z', -0.33], 'x': [-0.1675, 'z', 3.3325]}
lst_for_final_rezult [[-0.0047, 'z^3', 0.2805, 'z^2', -5.5805, 'z', 37.00927], [0.4489, 'z^2', -0.44, 'z', 0.1089], -14.0]


"ОТВЕТ: {1: {'z': 146.59944560365324, 'y': 146.9394456036532, 'x': 149.76444560365326}}"

In [31]:
system = [
    '- 8 + 4 x + y = 5',
    '2 x + z - y + 1 = 8',
    'x^2 + y^3 = 9 + 5',
]
solve(system)

[[1.0, 0.25, 0.0, -3.25], [0.0, -1.5, 1.0, -0.5]] 0
dict_for_substitution {'y': [0.67, 'z', -0.33], 'x': [-0.1675, 'z', 3.3325]}
lst_for_final_rezult [[0.0281, 'z^2', -1.12, 'z', 11.1056], [0.30076, 'z^3', -0.4444, 'z^2', 0.2189, 'z', -0.03594], -14.0]


"ОТВЕТ: {1: {'z': 2.767770917240528, 'y': 3.107770917240528, 'x': 5.9327709172405285}}"

In [32]:
s = [
    'x^2 + 4 x - y^4 = 7',
    'y + 5 z = - 9',
    'x = 9',
    'x - 12 y = 0'
]
solve(s)

'Колл-во переменных не совпадает с кол-вом уравнений'

In [33]:
s = [
    'y + 5 z = - 9',
    'x = 9',
    'z - 5 = 0'
]
solve(s)

[[1.0, 5.0, 0.0, 9.0], [0.0, 1.0, 0.0, -5.0], [0.0, 0.0, 1.0, -9.0]] 0
dict_for_substitution {'x': [9.0], 'z': [5.0], 'y': [-34.0]}
lst_for_final_rezult [0]


"ОТВЕТ: {'x': [9.0], 'z': [5.0], 'y': [-34.0]}"

In [34]:
s = [
    'x^2 + y - 2 = 9 ',
    'y = 9',
    'z - 5 = 0'
]
solve(s)

[[1.0, 0.0, -9.0], [0.0, 1.0, -5.0]] 0
dict_for_substitution {'z': [5.0], 'y': [9.0]}
lst_for_final_rezult ['x^2', [9.0], -11.0]


"ОТВЕТ: {1: {'x': 1.4142135623730951, 'z': 5.0, 'y': 9.0}, 2: {'x': -1.4142135623730951, 'z': 5.0, 'y': 9.0}}"

In [35]:
system = """- 8 + 4 x + y = 5
2 x + z - y + 1 = 8
x^3 + y^2 = 9 + 5"""


FILENAME = 'FOR CALCULATOR'

with open(FILENAME, "w") as file:                #сначала запишем нашу систему в файл
    print(system, file=file)

In [36]:
prob_list = []
probs = open(FILENAME, 'r')
for line in probs:
    prob_list.append(line)

if prob_list == []:
    prob_list = list(input('Введите список с системой внутри'))
else:
    solve(prob_list)

[[1.0, 0.25, 0.0, -3.25], [0.0, -1.5, 1.0, -0.5]] 0
dict_for_substitution {'y': [0.67, 'z', -0.33], 'x': [-0.1675, 'z', 3.3325]}
lst_for_final_rezult [[-0.0047, 'z^3', 0.2805, 'z^2', -5.5805, 'z', 37.00927], [0.4489, 'z^2', -0.44, 'z', 0.1089], -14.0]


In [37]:
system = ['- 8 + 4 x + y = 5','2 x + z - y + 1 = 8','x^3 + y^2 = 9 + 5']
solve(system)

[[1.0, 0.25, 0.0, -3.25], [0.0, -1.5, 1.0, -0.5]] 0
dict_for_substitution {'y': [0.67, 'z', -0.33], 'x': [-0.1675, 'z', 3.3325]}
lst_for_final_rezult [[-0.0047, 'z^3', 0.2805, 'z^2', -5.5805, 'z', 37.00927], [0.4489, 'z^2', -0.44, 'z', 0.1089], -14.0]


"ОТВЕТ: {1: {'z': 146.59944560365324, 'y': 146.9394456036532, 'x': 149.76444560365326}}"